In [25]:
from aiogram import Bot, Dispatcher, executor, types
from aiogram.types import ContentType
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.types.message import ContentTypes
from aiogram.dispatcher import FSMContext
from aiogram.contrib.fsm_storage.memory import MemoryStorage
from sqlalchemy import update

API_TOKEN = "5091011572:AAG4NfkC_zZjcsaAFkwLm4ZXOvhEqyLpQhY"


In [26]:
import asyncio

In [27]:
from food.paths import *
from food.search import *
import pandas as  pd
from food.psql import *
import pytz
timezones = pytz.all_timezones
import requests
from requests.structures import CaseInsensitiveDict
import urllib
from tzwhere import tzwhere

In [28]:
import nest_asyncio
nest_asyncio.apply()

In [29]:
def geocode(q):
    geocoding_key = '5d96ac126bcb462cb373297924ab2cb4'
    url = "https://api.geoapify.com/v1/geocode/search?"

    params = {"apiKey":geocoding_key, 
            "text":q}

    resp = requests.get(url + urllib.parse.urlencode(params)).json()
    return  pd.json_normalize(resp['features']).sort_values('properties.rank.importance',ascending = False)[['properties.lat','properties.lon']].iloc[0].to_list()
   

In [30]:
def get_tz(q):
    lat,lon = geocode(q)
    return tzwhere.tzwhere().tzNameAt(lat,lon)

In [31]:
async def async_get_tz(q):
    return get_tz(q)

In [32]:
async def async_search_image(url, env='prod'):
    return search_image(url,env)

In [33]:
async def async_geocode(q):
    return geocode(q)

In [34]:
async def  async_insert_on_conflict(*args, **qwargs):
    return insert_on_conflict(*args, **qwargs)


In [35]:
async def add_sender(message):
    sender = message['from'].to_python()
    sender = pd.DataFrame(sender,index=[0]).drop(columns =['is_bot'])
    await async_insert_on_conflict(sender,'users',unique_cols=['id'])

In [36]:
# Initialize bot and dispatcher
bot = Bot(token=API_TOKEN)
storage = MemoryStorage()
dp = Dispatcher(bot, storage=storage)

dishes_table = Dishes.__table__


In [37]:
ml_version = 0.2
"""0.2 - """

'0.2 - '

In [38]:
class CState(StatesGroup):
    init           = State()
    photo_taken    = State()
    measured       = State()
    set_timezone   = State()
    change_last     = State()
    change_weight  = State()
    

In [39]:
@dp.message_handler(commands=['start'])
async def send_welcome(message: types.Message):
    
    await CState.init.set()
    await message.reply("""Counting calories as easy as taking pictures. Just capture everything before you eat it\n
                          Now send a photo of your meal to try""")

In [40]:
@dp.message_handler(commands=['set_timezone'])
async def send_welcome(message: types.Message, state: FSMContext):
    await CState.set_timezone.set()
    await message.reply(f"please search your town to set timezone")


Polling is stopped.


In [41]:
@dp.message_handler(commands=['test'])
async def send_welcome(message: types.Message):

    reply_msg = types.message.Message(message_id=1931,from_user=message['from'])
    
    await reply_msg.reply("""test""")

In [42]:
@dp.message_handler(commands=['change_last_item'])
async def change_last(message: types.Message, state: FSMContext):
      print('change_last_item')
      global m
      m = message

      message_id = engine.execute(f"""select message_id from food.dishes 
                                    where user_id={message['from']['id']} and 
                                          grams >0
                                    order by id desc limit 1""").first()[0]

      async with state.proxy() as data: data['message_id'] = message_id


      reply_msg = types.message.Message(message_id = message_id,
                                          from_user  = types.user.User(id = message['from']['id']),
                                          chat       = types.chat.Chat(id = message['from']['id']))


      await CState.change_last.set()


      btns_text = tuple('change weight','remove','cancel')

      keyboard_markup = types.ReplyKeyboardMarkup(row_width=2)
      keyboard_markup.add(*(types.KeyboardButton(text) for text in btns_text))

      await reply_msg.reply("do you want to cansel mesurment for this item ?", reply_markup=keyboard_markup)           


In [43]:
@dp.message_handler(state=CState.change_last)
async def change_last_remove(message: types.Message, state: FSMContext):
    if message.text == 'remove':

        async with state.proxy() as data: message_id = data['message_id']

        stmt = (
                dishes_table.update()
                            .where(dishes_table.c.message_id == message_id)
                            .values(grams=0)
                            .returning(dishes_table.c.id)
        )
        session.execute(stmt)
        session.commit()                       
        await message.reply(""" your last item measurment has been removed""")

    elif message.text == 'change_weight':
        await CState.change_weight.set()

        btns_text = tuple([(str(p)) for p in range(10,510,10)])
        keyboard_markup = types.ReplyKeyboardMarkup(row_width=2)
        keyboard_markup.add(*(types.KeyboardButton(text) for text in btns_text))

        await message.reply("set weight for the dish", reply_markup=keyboard_markup) 

        

In [44]:
@dp.message_handler(lambda message: message.text.isdigit(),state=CState.change_weight)
async def change_last_change(message: types.Message, state: FSMContext):
    if message.text == 'change weight':
        
        async with state.proxy() as data: message_id = data['message_id']
        stmt = (
                dishes_table.update()
                            .where(dishes_table.c.message_id == message_id)
                            .values(grams=0)
                            .returning(dishes_table.c.id)
        )
        session.execute(stmt)
        session.commit()

                                    
        await message.reply(""" your last item measurment has been removed""")

In [45]:
@dp.message_handler(state=CState.set_timezone)
async def set_timezone(message: types.Message, state: FSMContext):
    await types.ChatActions.typing()
    await add_sender(message)
    tz = await async_get_tz(message.text)

    df = pd.DataFrame([[message['from']['id'],'tz',tz,pd.Timestamp.utcnow()]],columns = ['user_id','property','value','timestamp'])
    df.to_sql('user_properties',schema = schema,con = engine,if_exists = 'append',index = False)

    await message.reply(f"your tz is set to {tz}")

In [46]:
@dp.message_handler(content_types=ContentType.PHOTO)
async def process_photo(message: types.Message, state: FSMContext):
    global m
    m = message
    await types.ChatActions.typing()
    
    await add_sender(message)

    photo  = message['photo'][-1]
    await photo.download(reference_images_path/photo['file_id'])
    image_url = await photo.get_url()
    dish = await async_search_image(url=image_url, env='prod')
    
    dish['photo_id']   = photo['file_id']
    dish['message_id'] = message['message_id']
    sender = message['from'].to_python()
    dish['user_id'] = sender['id']
    dish['ml_version'] = ml_version 
    dish['timestamp']=pd.Timestamp.utcnow()


    # async with state.proxy() as data: data['dish'] = dish.to_dict(orient = 'records')[0]

    dish.to_sql('dishes',schema = schema,if_exists = 'append',index = False,con=engine)
    
    await CState.photo_taken.set()
    async with state.proxy() as data: data['photo_id'] = photo['file_id']

    btns_text = tuple([(str(p)) for p in range(10,510,10)])
    keyboard_markup = types.ReplyKeyboardMarkup(row_width=2)
    keyboard_markup.add(*(types.KeyboardButton(text) for text in btns_text))


    await message.reply("set weight of the dish you are going to eat", reply_markup=keyboard_markup)


In [47]:
@dp.message_handler(lambda message: message.text.isdigit(), state=CState.photo_taken)
async def measure(message: types.Message, state: FSMContext):

    grams = int(message.text)

    last_photo_id,energy = engine.execute(f"""select photo_id,energy from {schema}.dishes 
                                where user_id={message['from']['id']}
                                order by id desc limit 1""").first()
    # async with state.proxy() as data:  last_photo_id = data['photo_id']
    stmt = (
            dishes_table.update()
                        .where(dishes_table.c.photo_id == last_photo_id)
                        .values(grams=grams)
                        .returning(dishes_table.c.id)
    )
    session.execute(stmt)
    session.commit()

    # async with state.proxy() as data:  dish = data['dish']

    energy = energy/100*grams

    today_consumed = pd.read_sql(f"""select energy,grams,timestamp from {schema}.dishes
                                 where user_id = {message['from']['id']} and timestamp > now() - interval '24 hours'
                                 and grams is not null;""",engine).set_index("timestamp")
    today_consumed= today_consumed['energy']/100*today_consumed['grams']
    user_tz = engine.execute(f"""select value from food.user_properties 
                                where user_id={message['from']['id']} and
                                property='tz'
                                order by id desc limit 1""").first()

    user_tz = user_tz[0] if user_tz else 'UTC'
    today_consumed = today_consumed.tz_convert(user_tz)
    now = pd.Timestamp.now(tz = user_tz)
    today_consumed = today_consumed.reset_index()	
    this_morning = pd.Timestamp(year = now.year,month = now.month,day = now.day,hour = 3,tz = user_tz)
    today_consumed = today_consumed[today_consumed['timestamp'] > pd.Timestamp(this_morning)][0].sum()
    today_consumed


    await message.reply(f"""you have consumed {energy} ccal with whis dish \n
                          today consumed {today_consumed}""")

In [48]:
if __name__ == '__main__':
    executor.start_polling(dp)

Goodbye!


In [49]:
m

NameError: name 'm' is not defined